In [1]:
import scipy.stats as stats
import numpy as np
import mcmc

In [2]:
# school data
y = np.array([28.0, 8, -3, 7, -1, 1, 18, 12])
sigma = np.array([15, 10, 16, 11, 9, 11, 10, 18])

In [3]:
def log_density(theta):
    th = theta[0 : len(y)]
    mu = theta[-2]
    tau = theta[-1]
    
    if np.isnan(tau) or tau <= 0:
        return -np.inf
    else:
        log_hyperprior = 1
        log_prior = np.sum(stats.norm.logpdf(th, mu, tau))
        log_likelihood = np.sum(stats.norm.logpdf(y, th, sigma))
        return log_hyperprior + log_prior + log_likelihood

In [4]:
def gradient(theta):
    th = theta[0 : len(y)]
    mu = theta[-2]
    tau = theta[-1]
    J = len(y)
    
    if tau <= 0:
        return np.zeros(len(theta))
    else:
        d_theta = -(th - y) / sigma ** 2 - (th - mu) / tau ** 2
        d_mu = -np.sum((mu - th) / tau ** 2)
        d_tau = -J / tau + np.sum((mu - th) ** 2 / tau ** 3)
        return np.concatenate((d_theta, [d_mu], [d_tau]))

In [5]:
d = len(y) + 2
start = lambda: np.concatenate((stats.norm.rvs(0, 15, d - 1), stats.uniform.rvs(0, 15, 1)))
starts = [start() for _ in range(4)]
iterations = 2000 # must be divisible by 4
M = np.identity(d) / 15 ** 2
update = mcmc.hmc(log_density, gradient, M)
samples, rhat = mcmc.run(starts, iterations, update)
print np.round(rhat, 2)
samples.shape

[ 1.09  1.13  1.01  1.04  1.42  1.21  1.02  1.19  1.18  2.56]


mcmc.py:53: RuntimeWarning: overflow encountered in exp
  r = np.exp(logd(theta_star, phi_star) - logd(theta, phi))


(4000L, 10L)